In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from networkx.algorithms import community as cm

### Introdução 

Uma das tarefas comum em análise de redes complexas é agrupar os vértices da rede em subconjuntos, chamados de comunidades, de modo que os vértices mais similares fiquem juntos em uma comunidade. 

A ideia central é entender um pouco sobre como os nós tendem a se organizar dentro da rede através da observação dos padrões de ligação que os mesmos possuem.

Problemas de detecção de comunidades são similares a problemas de partição em grafos, mas em geral nestes últimos já há uma quantidade pré-determinada de grupos e dos tamanhos dos mesmos no qual o grafo deve ser particionado, o que não ocorre em geral com os problemas de detecção de comunidades nos quais o tamanho e a quantidade de comunidades a serem detectadas não são previamente conhecidos

### Modularidade

Modularidade equivale ao coeficiente de assortatividade não normalizado para atributos categóricos, sendo que aqui os atributos categóricos identificam as comunidades
aos quais os nós pertencem.

- **nx.karate_club_graph()** 

Returns Zachary's Karate Club graph.

Each node in the returned graph has a node attribute 'club' that indicates the name of the club to which the member represented by that node belongs, either 'Mr. Hi' or 'Officer'. 

Each edge has a weight based on the number of contexts in which that edge's incident node members interacted.

- **louvain_communities()**: 

Find the best partition of a graph using the Louvain Community Detection Algorithm.

Louvain Community Detection Algorithm is a simple method to extract the community structure of a network. 

This is a heuristic method based on modularity optimization.


- **nx.community.greedy_modularity_communities(G)** 

Find communities in G using greedy modularity maximization.

This function uses Clauset-Newman-Moore greedy modularity maximization to find the community partition with the largest modularity.

- **nx.modularity()**

Returns the modularity of the given partition of the graph.

In [ ]:
# Exemplo

G = nx.karate_club_graph()

fig, ax = plt.subplots(1,1,figsize=(15,10))

nx.draw(G,with_labels=True)

In [ ]:
# exemplo

comm = nx.community.louvain_communities(G, seed=123)
for it in comm:
    print(sorted(it))

In [ ]:
# exemplo

c = nx.community.greedy_modularity_communities(G)
for it in c:
    print(sorted(it))

In [ ]:
comsl = cm.louvain_communities(G)
comsc = cm.greedy_modularity_communities(G)

print('Modularidade - Louvain: {:.3f}'.format(cm.modularity(G,comsl)))
print('Modularidade - CNM    : {:.3f}'.format(cm.modularity(G,comsc)))

**Exemplo**

Considere o exemplo da rede dos personagens do segundo livro da saga de Harry Potter.
As ligações nessa rede representam suporte emocional e, portanto, trata-se de uma rede direcionada com matriz de adjacência não simétrica.

In [ ]:
atri = pd.read_csv('../data/harrypotter/hpattributes.txt', sep='\t')
ares = pd.read_csv('../data/harrypotter/hpbook2.txt', sep=' ', header=None)
nome = pd.read_csv('../data/harrypotter/hpnames.txt', sep='\t')

In [ ]:
# criando a rede

gpotter = nx.DiGraph()

n = atri.shape[0]

for k in range(n):
    gpotter.add_node(k,
                     nome = nome['name'][k],
                     ano = atri['schoolyear'][k],
                     gen = atri['gender'][k],
                     casa = atri['house'][k])

for k in range(n):
    for m in range(n):
        if ares.values[k][m] == 1:
            gpotter.add_edge(k,m)

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(15,10))

nx.draw(gpotter,with_labels=True)

In [ ]:
comsl = cm.louvain_communities(gpotter)
comsc = cm.greedy_modularity_communities(gpotter, cutoff=len(comsl))

print('Modularidade - Louvain: {:.3f}'.format(cm.modularity(gpotter,comsl)))
print('Modularidade - CNM    : {:.3f}'.format(cm.modularity(gpotter,comsc)))

In [ ]:
# matriz de adjacencia esparsa
A = nx.adjacency_matrix(gpotter)

# apagando os nos isolados
isolados = []
for k in range(n):
    if np.sum(A[[k],:]) == 0:
        isolados.append(k)
        gpotter.remove_node(k)

print(len(isolados), 'nos isolados.')

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(15,10))

nx.draw(gpotter,with_labels=True)

Para comparar os resultados, no algoritmo de **Clauset-Newman-Moore** só salvamos o mesmo número de comunidades que as devolvidas pelo algoritmo de **Louvain**. 

In [ ]:
comsl = cm.louvain_communities(gpotter)
comsc = cm.greedy_modularity_communities(gpotter,cutoff=len(comsl))

print('Modularidade - Louvain: {:.3f}'.format(cm.modularity(gpotter,comsl)))
print('Modularidade - CNM    : {:.3f}'.format(cm.modularity(gpotter,comsc)))

In [ ]:
for it in comsl:
    print(sorted(it))

In [ ]:
for it in comsc:
    print(sorted(it))

In [ ]:
plt.close('all')

pos = nx.circular_layout(gpotter)

coresp = {0:'lightskyblue',
          1:'steelblue',
          2:'gray',
          3:'pink',
          4:'magenta',
          5:'violet',
          6: 'mediumorchid'}

coresl = []
for no in gpotter.nodes():
    for k in range(len(comsl)):
        if no in comsl[k]:
            coresl.append(coresp[k])
            break

coresc = []
for no in gpotter.nodes():
    for k in range(len(comsc)):
        if no in comsc[k]:
            coresc.append(coresp[k])
            break
    
etiquetas = nx.get_node_attributes(gpotter, 'nome')

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(15,10))

nx.draw_networkx(gpotter,
                 pos=pos,
                 edge_color='lightgray',
                 node_size = 500,
                 labels=etiquetas,
                 font_size=10, 
                 node_color=coresl)

plt.box(False)
plt.show()

In [ ]:
#plt.figure()

fig, ax = plt.subplots(1,1,figsize=(15,10))

nx.draw_networkx(gpotter,
                 pos=pos,
                 edge_color='lightgray',
                 node_size = 500,
                 labels=etiquetas,
                 font_size=10, 
                 node_color=coresc)

plt.box(False)
plt.show()

Considere o dataset **Stack Overflow Tag Network**, um ***Network (links and nodes) of Stack Overflow tags based on Developer Stories***

Link: https://www.kaggle.com/datasets/stackoverflow/stack-overflow-tag-network



In [ ]:
df = pd.read_csv('../data/stack_network/stack_network_links.csv')

In [ ]:
df.head()

In [ ]:
G = nx.Graph()

In [ ]:
with open('../data/stack_network/stack_network_links.csv') as f:
  next(f)
  G = nx.parse_edgelist(f.readlines(), comments='/', delimiter=',', data=(('weight', float),))

In [ ]:
list(G.nodes())[:5]

In [ ]:
list(G.edges.data('weight'))[:5]

In [ ]:
communities = cm.greedy_modularity_communities(G, weight='weight')

In [ ]:
len(communities)

In [ ]:
for it in communities:
    print(it)

In [ ]:
import pygraphviz

plt.figure(figsize=(20,14))

nx.draw(G, pos = nx.nx_agraph.graphviz_layout(G), \
    node_size=200, node_color='lightblue', linewidths=1.5, \
    font_size=11, font_weight='bold', with_labels=True)

In [ ]:
communities_dict = {}

for i in range(len(communities)):
  for node in list(communities[i]):
    communities_dict[node] = i

nx.set_node_attributes(G, communities_dict, name='group')

In [ ]:
for key, value in communities_dict.items():
    print(key, value, end=", ")

In [ ]:
len(communities_dict)

In [ ]:
from pyvis.network import Network

net = Network(notebook=True, cdn_resources='in_line')
_ = net.from_nx(G)
_ = net.show('communities.html')